In [1]:
import os
import random
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import ElectraModel, ElectraTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

/opt/tljh/user/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-20 09:43:55.880567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 09:43:58.922687: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load BERT and Tokenizer

In [2]:
bert = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
tokenizer = ElectraTokenizer.from_pretrained('tokenizer')

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Label Dicts

In [4]:
labels = ['[PAD]', 'E_B', 'E_I', 'O']
num_labels = len(labels)
id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in id2label.items()}

## Load Data and Preprocess for Training

In [5]:
data = pd.read_pickle('data/preprocessed.pkl')

In [6]:
data['tokens'] = data.tokens.apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
data['labels'] = data.labels.apply(lambda x: ['O'] + x + ['O'])

In [7]:
data.tokens.apply(lambda x: len(x)).max()

233

In [8]:
max_len = 256

In [9]:
data['tokens'] = data.tokens.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))
data['labels'] = data.labels.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))

In [10]:
tokens_lst = data.tokens.to_list()
labels_lst = data.labels.to_list()

In [11]:
X_train, X_eval, y_train, y_eval = train_test_split(tokens_lst, 
                                                    labels_lst, 
                                                    test_size=0.2, shuffle=True, random_state=42)

In [12]:
train_data = []
for tokens, labels in zip(X_train, y_train):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)
    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    train_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])

In [13]:
eval_data = []
for tokens, labels in zip(X_eval, y_eval):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)
    
    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    eval_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])

In [14]:
# idx = random.randrange(0, len(train_data) - 1)
# for x, xm, y in zip(train_data[idx][0], train_data[idx][1], train_data[idx][2]):
#     print(x, xm, y)

# idx = random.randrange(0, len(eval_data) - 1)
# for x, xm, y in zip(eval_data[idx][0], eval_data[idx][1], eval_data[idx][2]):
#     print(x, xm, y)

## HP Config

In [15]:
batch_size = 128
LEARNING_RATE = 5e-5
N_EPOCHS = 15

## Create Dataset and Generate Dataloader

In [16]:
class TaggerDataset(Dataset): 
    def __init__(self, data):
        self.data = data
    
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.data[idx][0]
        mask = self.data[idx][1]
        label_ids = self.data[idx][2]
        return (torch.LongTensor(input_ids), torch.LongTensor(mask), torch.LongTensor(label_ids))

In [17]:
train_dataset = TaggerDataset(train_data)
eval_dataset = TaggerDataset(eval_data)

In [18]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
eval_loader = DataLoader(eval_dataset, batch_size = batch_size, shuffle = True)

## Instantiate Model

In [19]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text, mask):
        embedded = self.dropout(self.bert(text, mask)[0])
        predictions = self.fc(self.dropout(embedded))
        return predictions

In [20]:
OUTPUT_DIM = num_labels
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

Embedding(36223, 768)

In [21]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = 0)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=0, factor=0.7, min_lr=0)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))
    # model = torch.nn.DataParallel(model, device_ids=[0,1])
    # torch.multiprocessing.set_start_method('spawn', force=True)
model = model.to(device)
criterion = criterion.to(device)

In [23]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])

    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [24]:
def categorical_f1(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    max_preds_no_pad = max_preds[non_pad_elements].squeeze(1).detach().cpu()
    y_no_pad = y[non_pad_elements].detach().cpu()
    
    f1_macro = f1_score(y_no_pad, max_preds_no_pad, average='macro')
    f1_micro = f1_score(y_no_pad, max_preds_no_pad, average='micro')    
    
    return f1_macro, f1_micro

In [25]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    for batch in iterator:
        text = batch[0].to(device)
        mask = batch[1].to(device)
        tags = batch[2].to(device)

        predictions = model(text, mask)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        if predictions_set == None:
            predictions_set = predictions
            tags_set = tags
        else:
            predictions_set = torch.cat([predictions_set, predictions], dim=0)
            tags_set = torch.cat([tags_set, tags], dim=0)
        
        loss = criterion(predictions, tags)
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

In [26]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    with torch.no_grad():
        for batch in iterator:
            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            if predictions_set == None:
                predictions_set = predictions
                tags_set = tags
            else:
                predictions_set = torch.cat([predictions_set, predictions], dim=0)
                tags_set = torch.cat([tags_set, tags], dim=0)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

In [27]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1_mac, train_f1_mic = train(model, train_loader, optimizer, criterion, 0)
    valid_loss, valid_acc, valid_f1_mac, valid_f1_mic = evaluate(model, eval_loader, criterion, 0)
    
    cur_lr = scheduler.optimizer.state_dict()['param_groups'][0]['lr']
    scheduler.step(valid_loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'event_tagger.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'Learning Rate: {cur_lr}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%', end=' | ')
    print(f'Train F1 Mac: {train_f1_mac*100:.2f}% | Train F1 Mic: {train_f1_mic*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%', end=' |  ')
    print(f'Val. F1 Mac: {valid_f1_mac*100:.2f}% |  Val. F1 Mic: {valid_f1_mic*100:.2f}%', end='\n\n')

Epoch: 01 | Epoch Time: 1m 5s
Learning Rate: 5e-05
	Train Loss: 0.810 | Train Acc: 71.30% | Train F1 Mac: 23.43% | Train F1 Mic: 71.47%
	 Val. Loss: 0.660 |  Val. Acc: 77.64% |  Val. F1 Mac: 29.14% |  Val. F1 Mic: 77.65%

Epoch: 02 | Epoch Time: 0m 17s
Learning Rate: 5e-05
	Train Loss: 0.639 | Train Acc: 77.07% | Train F1 Mac: 30.97% | Train F1 Mic: 76.81%
	 Val. Loss: 0.524 |  Val. Acc: 80.43% |  Val. F1 Mac: 48.50% |  Val. F1 Mic: 80.49%

Epoch: 03 | Epoch Time: 0m 17s
Learning Rate: 5e-05
	Train Loss: 0.519 | Train Acc: 79.96% | Train F1 Mac: 54.37% | Train F1 Mic: 79.87%
	 Val. Loss: 0.494 |  Val. Acc: 81.70% |  Val. F1 Mac: 55.10% |  Val. F1 Mic: 81.66%

Epoch: 04 | Epoch Time: 0m 17s
Learning Rate: 5e-05
	Train Loss: 0.429 | Train Acc: 83.70% | Train F1 Mac: 66.76% | Train F1 Mic: 83.47%
	 Val. Loss: 0.333 |  Val. Acc: 87.32% |  Val. F1 Mac: 74.38% |  Val. F1 Mic: 87.35%

Epoch: 05 | Epoch Time: 0m 17s
Learning Rate: 5e-05
	Train Loss: 0.324 | Train Acc: 87.74% | Train F1 Mac: 75